# TESTING MODEL

In [1]:
import os
import warnings
import csv

# Import .py files
from dicom_utils import *
from helper import *

warnings.filterwarnings('ignore') # ignore tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # ignore tensorflow warnings

# Import PyTorch
import torch

# Import monai model
from monai.networks.nets import resnet10

In [2]:
# Set directory to root
os.chdir('/')
data_dir = "data/classification" 
save_dir = "group/bagel/Task_2/Final" 
csv_path = f"{data_dir}/example_submission.csv"

# Model variables
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [4]:
# Load FLAIR Model
f_model_path = f"{save_dir}/models/FLAIR_model"
f_model = resnet10(spatial_dims=3, n_input_channels=1, num_classes=1)
fmodel, f_model_summary = load_model(f_model, f_model_path)
set_random_seed(42)
f_test_dl = get_test_data_generators(csv_path, 'FLAIR', f_model_summary['num_3d_images'], 1)
f_predictions = get_prediction(f_model, f_test_dl, device)

100%|██████████| 117/117 [02:59<00:00,  1.54s/it]


In [5]:
# Load T1wCE Model
t_model_path = f"{save_dir}/models/FLAIR_model"
t_model = resnet10(spatial_dims=3, n_input_channels=1, num_classes=1)
t_model, t_model_summary = load_model(t_model, t_model_path)
set_random_seed(42)
t_test_dl = get_test_data_generators(csv_path, 'T1wCE', f_model_summary['num_3d_images'], 1)
t_predictions = get_prediction(t_model, t_test_dl, device)

100%|██████████| 117/117 [03:04<00:00,  1.58s/it]


In [31]:
list_id = list(pd.read_csv(csv_path, sep=';')['ID'])

predictions = { "ID": [],
                "MGMT_probability": [],
                "MGMT_value": []}
for i in list_id:
    predictions["ID"].append(i)
    predictions["MGMT_probability"].append(0.7*f_predictions[i]+0.3*t_predictions[i])
    predictions["MGMT_value"].append(int((f_predictions[i]) > (f_model_summary['threshold'])))
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv(f"{save_dir}/submission.csv", index=False)

python submission/evaluation_script/evaluate.py --task classification --true-path group/bagel/Task_2/Final --pred-path group/bagel/Task_2/Final

In [38]:
list_id = list(pd.read_csv(csv_path, sep=';')['ID'])
true = { "ID": [], "MGMT_value": []}
p = 0
for i in list_id:
    true["ID"].append(i)
    true["MGMT_value"].append(int(p))
    p = (p+1)%2
true_df = pd.DataFrame(true)
true_df.to_csv(f"{save_dir}/classification/labels.csv", index=False)